In [1]:
from numpy.random import seed
import tensorflow as tf
import os
from data_preprocessing import preprocessing,train_test
from models import model_builder,model_fit
from evaluation import generate_test_data,mdn_auc_score,generate_csv

In [2]:
input_file = '../sample_dataset/dummyservice_90percentile_1804-3006_predwindow_1_mdn_1.csv' #Location of dataset csv
timesteps = 1 #Timesteps of the input data to be considered.
split = 0.2 #Split for test-train data
components = 3 #Number of gaussian distributions to be used.
neurons = 200 #Number of neurons in the mdn layer.
lstm_variant='bilstm' #Temporal model to be used, Available options: simpleRNN/gru/stacked_lstm/lstm/stacked_bilstm/bilstm
epoch_cnt = 1 #Epoch counts for model run 
batch_size =  32 #Batch size
no_of_metrics = 7 #Number of metrics predicting
tgt_dir = 'results/' #Target directory where to save results data
file_det = 'dummyservice_90percentile_1804-3006_predwindow_1_' #Target file to save data
classifier_layers = ['istio_request_duration_milliseconds_count_5xx_pred',
 'apm.service.cpu.usertime.utilization_pred',
 'elasticsearch_os_load15_pred',
 'apm.service.apdex_pred',
 'apm.service.error.count_pred',
 'throughput_pred',
 'istio_request_duration_milliseconds_count_4xx_pred'] # Name of columns containing classification data
file_pred = '../sample_dataset/dummyservice_95percentile_1804-3006_predwindow_1_mdn_1.csv' #Location of csv to test
threshold_percentage = 95 #Threshold percentage of the file we are testing.
pred_window = 1 #Prediction window of the file (should be same for train data and testing data)
tails =  {'istio_request_duration_milliseconds_count_5xx': 'r',
          'apm.service.cpu.usertime.utilization': 'r',
          'elasticsearch_os_load15': 'r',
          'apm.service.error.count': 'r',
          'throughput': 'l',
          'istio_request_duration_milliseconds_count_4xx': 'r',
          'apm.service.apdex': 'l'} #Tails of the metrics

if not os.path.exists(tgt_dir):
    os.makedirs(tgt_dir)

### Model Run

In [3]:
import warnings
warnings.filterwarnings("ignore")
seed_value= 42 
seed(seed_value)
tf.random.set_seed(seed_value)


#Preprocessing

input_and_labels,classifier_layers,mdn_layers=preprocessing(input_file)
trainX,testX,y_train_classifier,y_test_classifier,y_train_mdn,y_test_mdn,scalerY=train_test(input_and_labels,
                                                                                            classifier_layers,
                                                                                            mdn_layers,
                                                                                            timesteps,
                                                                                            split)

#Model generation
model=model_builder(trainX,classifier_layers,mdn_layers,lstm_variant,neurons,components)

#Model run
history = model_fit(model,trainX,testX,y_train_classifier,y_test_classifier,y_train_mdn,y_test_mdn,epoch_cnt,batch_size)

#Evaluation
y_test,test = generate_test_data(file_pred,classifier_layers,split,timesteps)
df_prob,thresholds = mdn_auc_score(input_and_labels,
                                   tgt_dir,
                                   file_det,
                                   scalerY,
                                   mdn_layers,
                                   model,
                                   testX,
                                   y_test,
                                   threshold_percentage,
                                   pred_window,
                                   tails,
                                  no_of_metrics,components)
# #Dashboard data
generate_csv(test,df_prob,classifier_layers,mdn_layers,thresholds,tgt_dir,file_det,timesteps)

533/533 [==============================] - 26s 17ms/step - loss: 11.3915 - output_istio_request_duration_milliseconds_count_5xx_pred_loss: 0.1331 - output_apm.service.cpu.usertime.utilization_pred_loss: 0.2755 - output_elasticsearch_os_load15_pred_loss: 0.2544 - output_apm.service.apdex_pred_loss: 0.1708 - output_apm.service.error.count_pred_loss: 0.1838 - output_throughput_pred_loss: 0.3312 - output_istio_request_duration_milliseconds_count_4xx_pred_loss: 0.0859 - output_istio_request_duration_milliseconds_count_5xx_forecast_loss: 0.9350 - output_apm.service.cpu.usertime.utilization_forecast_loss: 0.9317 - output_elasticsearch_os_load15_forecast_loss: 0.9357 - output_apm.service.apdex_forecast_loss: 0.9354 - output_apm.service.error.count_forecast_loss: 0.9388 - output_throughput_forecast_loss: 0.9397 - output_istio_request_duration_milliseconds_count_4xx_forecast_loss: 0.9358 - output_istio_request_duration_milliseconds_count_5xx_pred_auc: 0.5903 - output_apm.service.cpu.usertime.uti